In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
from datetime import datetime

import pandas as pd

from src.get_data import (
    get_coinmetrics_data,
    get_metrics_names
)

In [3]:
# Paths
dump_data_path = '../data/dynamic/eth_dump.csv'
coinmetrics_data_path = '../data/dynamic/eth_coinmetrics_20150730_20240121.csv'
file_path_metrics = '../data/static/metrics.txt'
file_path_assets_eth = '../data/static/assets_eth.txt'

# 1.Get Data

## 1.1. ETH dump

In [4]:
# Import dump data
dump_data = pd.read_csv(dump_data_path)

## 1.2. Coinmetrics data

In [13]:
# Check if the file exist
load_local_coinmetrics_data = False
if load_local_coinmetrics_data:
    coinmetrics_data = pd.read_csv(coinmetrics_data_path)
else:
    # Compute duration depth
    first_date = dump_data['time'].iloc[0]
    last_date = dump_data['time'].iloc[-1]
    # Compute depth
    def days_between(d1, d2):
        d1 = datetime.strptime(d1, "%Y-%m-%d")
        d2 = datetime.strptime(d2, "%Y-%m-%d")
        return abs((d2 - d1).days)
    depth_dump_data = days_between(first_date, last_date)
    depth_now_last_date_dump = days_between(last_date, datetime.now().strftime("%Y-%m-%d"))
    depth = depth_dump_data + depth_now_last_date_dump
    # Get coinmetrics data to fetch
    days_before_today = depth
    coinmetrics_data = (
        get_coinmetrics_data(
            days_before_today,
            file_path_metrics,
            file_path_assets_eth
        )
    )
    # Save fetched data
    coinmetrics_data.to_csv(coinmetrics_data_path, index=False)

2024-01-21 15:40:32 INFO     API key not found. Using an empty string.
2024-01-21 15:40:46 INFO     Sleeping for a rate limit window because 429 (too many requests) error was returned. Pleasesee Coin Metrics APIV4 documentation for more information: https://docs.coinmetrics.io/api/v4/#tag/Rate-limits
c:\Users\HP-PC-MKA\Documents\Repositories\Robinhood\envs\env_rb\lib\site-packages\pandas\core\dtypes\cast.py:1056: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():
c:\Users\HP-PC-MKA\Documents\Repositories\Robinhood\envs\env_rb\lib\site-packages\pandas\core\dtypes\cast.py:1080: RuntimeWarning: invalid value encountered in cast
  if (arr.astype(int) == arr).all():


# 2. Process Data

In [14]:
# Get metrics
metrics = get_metrics_names(file_path_metrics)

In [15]:
# Select metrics columns
dump_data_selected = dump_data.loc[:,['time'] + metrics]
coinmetrics_data_selected = coinmetrics_data.loc[:,['time'] + metrics]

In [16]:
# Remove data before last date
last_date = dump_data['time'].iloc[-1]
coinmetrics_data_selected = (
    coinmetrics_data_selected[
        coinmetrics_data_selected['time'] < last_date
    ]
)
dump_data_selected = (
    dump_data_selected[
        dump_data_selected['time'] < last_date
    ]
)

In [17]:
# Fill Na with 0 of both dataframes
dump_data_selected_filled = dump_data_selected.fillna(0)
coinmetrics_data_selected_filled = coinmetrics_data_selected.fillna(0)

In [18]:
difference = (
    dump_data_selected_filled
    .drop(columns=['time'])
    .sub(coinmetrics_data_selected_filled.drop(columns=['time']))
)

In [19]:
difference

,AdrActCnt,AdrBalCnt,AssetEODCompletionTime,BlkCnt,BlkSizeMeanByte,CapAct1yrUSD,CapMVRVCur,CapMVRVFF,CapMrktCurUSD,CapMrktFFUSD,...,TxCnt,TxCntSec,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,VelCur1yr
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3048,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
difference.sum().sum()

0.0

**Comment**
- No difference between both datasets.